In [1]:
import napari
from skimage import data
import numpy as np
from skimage import measure, morphology, filters
import pandas as pd
from typing import List, Optional, Tuple

import matplotlib.colors as mcolor
import napari
import numpy as np
import pandas as pd
from magicgui import magicgui
from magicgui.widgets import ComboBox
import matplotlib.pyplot as plt
import seaborn as sns

from napari_matplotlib.util import Interval
from napari_matplotlib.line import Line2DBaseWidget


In [2]:
image = data.human_mitosis()
nrows, ncols = image.shape

In [3]:
image1 = image[:nrows//2, :ncols//2]
image2 = image[:nrows//2, ncols//2:]
image3 = image[nrows//2:, :ncols//2]
image4 = image[nrows//2:, ncols//2:]
image_list = [image1, image1, image1, image1]

Create Label Images

In [4]:
labels_list = []
for image in image_list:
    filtered = filters.gaussian(image, 1)
    th = filters.threshold_otsu(filtered)
    binary = filtered > th
    labels = measure.label(binary)
    labels_list.append(labels)

In [5]:
image_list = [image1, 2*image1, 3*image1, 4*image1]

Feature Extraction

In [6]:
tables_list = []
for labels, image, i in zip(labels_list, image_list, range(len(image_list))):
    table = pd.DataFrame(measure.regionprops_table(labels, image, properties=['label', 
                                                                                'area', 
                                                                                'axis_major_length', 
                                                                                'axis_major_length', 
                                                                                'feret_diameter_max', 
                                                                                'orientation', 
                                                                                'solidity', 
                                                                                'eccentricity', 
                                                                                'intensity_mean',
    ]))
    table['frame'] = i+1
    tables_list.append(table)

concat tables

In [7]:
master_table = pd.concat(tables_list)
master_table

,label,area,axis_major_length,feret_diameter_max,orientation,solidity,eccentricity,intensity_mean,frame
0,1,64,10.862780,10.770330,1.570796,1.000000,0.713074,50.250000,1
1,2,73,12.568236,13.000000,0.507038,0.935897,0.808680,59.054795,1
2,3,77,11.542712,11.704700,0.041654,0.987179,0.680056,52.558442,1
3,4,86,11.624222,12.083046,-1.391353,0.945055,0.570413,47.790698,1
4,5,137,13.928775,14.866069,1.338121,0.951389,0.435103,56.416058,1
...,...,...,...,...,...,...,...,...,...
43,44,91,12.076837,12.649111,0.396809,0.947917,0.598185,163.428571,4
44,45,86,11.446477,12.083046,-1.105524,0.945055,0.548494,191.906977,4
45,46,127,14.118014,14.560220,0.108927,0.969466,0.582361,106.047244,4
46,47,60,11.865918,11.401754,0.000000,1.000000,0.834639,123.733333,4


Stack labels

In [8]:
labels_stack = np.stack(labels_list)
labels_stack = labels_stack[:, np.newaxis]
labels_stack.shape

(4, 1, 256, 256)

Open in napari

In [9]:
viewer = napari.Viewer()
viewer.add_labels(labels_stack, features=master_table)

<Labels layer 'labels_stack' at 0x18abcaff250>

In [10]:
from napari_skimage_regionprops._table import add_table

In [12]:
table_widget = add_table(viewer.layers[-1], viewer)

Napari status bar display of label properties disabled because https://github.com/napari/napari/issues/5417 and https://github.com/napari/napari/issues/4342


[] <class 'list'> 0
[0      1
1      1
2      1
3      1
4      1
      ..
187    4
188    4
189    4
190    4
191    4
Name: frame, Length: 192, dtype: int64, 0       1
1       2
2       3
3       4
4       5
       ..
187    44
188    45
189    46
190    47
191    48
Name: index, Length: 192, dtype: int32, 0       1
1       2
2       3
3       4
4       5
       ..
187    44
188    45
189    46
190    47
191    48
Name: label, Length: 192, dtype: int32] <class 'list'> 3


In [27]:
def _on_rightclick_multi_column(self):

    if not 'label' in self._table.keys() and not 'frame' in self._table.keys():
        return
    
    selected_column = list(self._table.keys())[self._view.currentColumn()]

    widget = FeaturesLine2DWidget(viewer, show_dropdowns=False)
    self._viewer.window.add_dock_widget(widget, tabify=True)
    widget._set_axis_keys('frame', selected_column)


In [ ]:

table_widget._view.horizontalHeader().setContextMenuPolicy
